In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from collections import Counter
import pickle

In [26]:
# Charger les données
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_data_dict = pd.read_excel('data dictionary.xlsx')

# Définir les colonnes à sélectionner pour l'entraînement du modèle
features = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Fuel_Type', 'Transmission', 'Owner_Type']


In [27]:
# Vectoriser la colonne "Name" en utilisant TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train['Name'])

# Appliquer le clustering k-means
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)

# Ajouter les étiquettes de cluster à la colonne "Name"
df_train['Name_Cluster'] = kmeans.labels_

# Attribuer les noms de clusters les plus fréquents
cluster_names = []
for i in range(10):
    names = df_train['Name'][df_train['Name_Cluster'] == i]
    counter = Counter(names)
    cluster_name = counter.most_common(1)[0][0]
    cluster_names.append(cluster_name)

# Remplacer les valeurs dans la colonne "Name_Cluster" par les nouveaux noms de cluster
cluster_map = dict(zip(range(10), cluster_names))
df_train['Name_Cluster'] = df_train['Name_Cluster'].map(cluster_map)


In [28]:
# Nettoyer et préparer les données
df_train['Engine'] = df_train['Engine'].str.replace(' CC', '', regex=False)
df_train['Mileage'] = df_train['Mileage'].str.replace(' kmpl| km/kg', '', regex=True).astype(float)
df_train['Power'] = df_train['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)
df_train = df_train.drop(['New_Price'], axis=1)

num_cols = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats']
cat_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Name_Cluster']

# Séparer les données en features et cible
X = df_train.drop(['Price'], axis=1)
y = df_train['Price']

In [29]:
# Définir les transformations pour les colonnes numériques et catégorielles
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combiner les transformations en une seule étape de prétraitement
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Définir le modèle de régression
model = GradientBoostingRegressor(random_state=42)

# Créer la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [30]:
# Diviser les données d'entraînement en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle sur les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluer les performances du modèle sur les données de validation
score = pipeline.score(X_val, y_val)
print('Score de la pipeline sur les données de validation :', score)

# Faire des prédictions sur les données de validation
y_pred = pipeline.predict(X_val)

# Coefficient de détermination (R²)
r2 = r2_score(y_val, y_pred)
print("R² :", r2)

# Erreur moyenne absolue (MAE)
mae = mean_absolute_error(y_val, y_pred)
print("MAE :", mae)

# Erreur quadratique moyenne (MSE)
mse = mean_squared_error(y_val, y_pred)
print("MSE :", mse)

# Erreur quadratique moyenne racine (RMSE)
rmse = np.sqrt(mse)
print("RMSE :", rmse)

Score de la pipeline sur les données de validation : 0.8529797697028105
R² : 0.8529797697028105
MAE : 2.0278836367145616
MSE : 18.09223483839815
RMSE : 4.253496777758054


In [31]:
# Enregistrer le modèle entraîné sous forme de fichier pickle
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
